# 09 — FastAPI API: Consumo, Testes e Automação (Bilingue)

Este notebook ensina como consumir, testar e automatizar chamadas à API FastAPI deste projeto.
This notebook teaches how to consume, test, and automate calls to the project's FastAPI API.

Conteúdo / Contents:
- Setup de ambiente (requests, httpx, pytest, locust opcional)
- Endpoints: predição (sync/async), batch, explicabilidade (SHAP/LIME), saúde/versão
- Autenticação (Bearer/JWT) e variáveis de ambiente
- Testes automatizados de contrato e integração
- Integração com front-end (Streamlit)
- Automação e agendamento (Prefect/Airflow)


In [ ]:
# %% [markdown]
# 🔧 Setup / Instalação
# Execute uma vez para garantir dependências.
# Run once to ensure dependencies.
!pip -q install requests httpx python-dotenv pytest rich >/dev/null 2>&1
# opcional / optional: 
# !pip -q install locust shap lime prefect apache-airflow >/dev/null 2>&1


## 🔑 Configuração / Configuration
Defina a URL base da API e o token (se houver) via .env.
Define API base URL and token (if any) via .env.


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
API_BASE = os.getenv('API_BASE', 'http://localhost:8000')
API_TOKEN = os.getenv('API_TOKEN')  # ex: JWT Bearer
headers = {'Authorization': f'Bearer {API_TOKEN}'} if API_TOKEN else {}
print('API_BASE =', API_BASE)
print('Auth =', 'Bearer' if API_TOKEN else 'None')


## ✅ Health/Version
Checar se a API está viva e obter versão.
Check if API is alive and get version.


In [ ]:
import requests
r = requests.get(f'{API_BASE}/health')
print('Health:', r.status_code, r.json() if r.headers.get('content-type','').startswith('application/json') else r.text)
r = requests.get(f'{API_BASE}/version')
print('Version:', r.status_code, r.json() if r.headers.get('content-type','').startswith('application/json') else r.text)


## 🧠 Predição / Prediction
Exemplo de payload conforme o schema do projeto.
Example payload according to the project's schema.


In [ ]:
sample = {
    'features': {
        'age': 42,
        'income': 52000,
        'city': 'Sao Paulo'
    }
}
r = requests.post(f'{API_BASE}/predict', json=sample, headers=headers)
print(r.status_code, r.json())


## 📦 Batch Prediction
Envie múltiplos registros de uma vez.
Send multiple records at once.


In [ ]:
batch = {
  'items': [
    {'features': {'age': 30, 'income': 30000, 'city': 'Rio'}},
    {'features': {'age': 55, 'income': 90000, 'city': 'Sao Paulo'}}
  ]
}
r = requests.post(f'{API_BASE}/predict/batch', json=batch, headers=headers)
print(r.status_code, r.json())


## 🧾 Explicabilidade / Explainability
Obtenha explicações SHAP/LIME se expostas no endpoint.
Get SHAP/LIME explanations if endpoint is available.


In [ ]:
r = requests.post(f'{API_BASE}/explain', json=sample, headers=headers)
print(r.status_code)
try:
    print(r.json())
except Exception:
    print(r.text[:500])


## 🔐 Autenticação / Authentication
Fluxo típico: obter token via /auth/login, usar como Bearer.
Typical flow: get token via /auth/login, use as Bearer.


In [ ]:
# Example only — adjust to your API.
creds = {'username': os.getenv('API_USER'), 'password': os.getenv('API_PASS')}
if creds['username'] and creds['password']:
    r = requests.post(f'{API_BASE}/auth/login', json=creds)
    if r.ok:
        token = r.json().get('access_token')
        headers.update({'Authorization': f'Bearer {token}'})
        print('Token acquired.')
    else:
        print('Login failed:', r.status_code, r.text)
else:
    print('Set API_USER and API_PASS in .env to demo login flow.')


## 🧪 Testes Automatizados / Automated Tests
Testes de contrato e integração com pytest/httpx.
Contract and integration tests with pytest/httpx.


In [ ]:
%%bash
cat > tests_api.py << 'PY'
import os, httpx
API_BASE = os.getenv('API_BASE', 'http://localhost:8000')
def test_health():
    r = httpx.get(f'{API_BASE}/health', timeout=10)
    assert r.status_code == 200
def test_predict_contract():
    payload = {'features': {'age': 25, 'income': 40000, 'city': 'BH'}}
    r = httpx.post(f'{API_BASE}/predict', json=payload, timeout=15)
    assert r.status_code == 200
    j = r.json()
    assert 'prediction' in j or 'proba' in j
PY
pytest -q tests_api.py || true


## 🖥️ Integração com Front-end (Streamlit)
Exemplo simples chamando a API a partir de uma UI.
Simple example calling API from a UI.


In [ ]:
%%bash
cat > app_streamlit.py << 'PY'
import os, requests, streamlit as st
API_BASE = os.getenv('API_BASE', 'http://localhost:8000')
st.title('FastAPI Demo')
age = st.number_input('Age', 0, 120, 30)
income = st.number_input('Income', 0, 1000000, 50000)
city = st.text_input('City', 'Sao Paulo')
if st.button('Predict'):
    r = requests.post(f'{API_BASE}/predict', json={'features': {'age': age, 'income': income, 'city': city}})
    st.write(r.json())
PY
echo 'Run: streamlit run app_streamlit.py'


## ⏱️ Automação (Prefect/Airflow)
Orquestre execuções de batch com Prefect ou Airflow.
Orchestrate batch runs with Prefect or Airflow.


In [ ]:
# Exemplo Prefect / Prefect example
from datetime import datetime
try:
    from prefect import flow, task
    import requests as rq
    @task
    def call_batch():
        payload = {'items': [{'features': {'age': 20, 'income': 20000, 'city': 'Recife'}}]}
        res = rq.post(f'{API_BASE}/predict/batch', json=payload)
        return res.status_code
    @flow(name='api_batch_flow')
    def api_batch_flow():
        sc = call_batch()
        print('Batch status:', sc)
    if __name__ == '__main__':
        api_batch_flow()
except Exception as e:
    print('Prefect not installed or example failed:', e)


---
Dicas:
- Use variáveis de ambiente para URLs/tokens.
- Documente o schema dos endpoints no README.
- Adicione testes ao CI para validar a API a cada commit.
Tips:
- Use environment variables for URLs/tokens.
- Document endpoint schemas in README.
- Add tests to CI to validate API on each commit.
